# Find KEGG associations

This notebook will create a table that has the KEGG pathways that are associated with all genes, but we are particularly interested in those that are associated with the most and least stable genes.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import random
import pandas as pd
from scripts import paths, utils, modules, annotations

random.seed(1)

In [2]:
# Output files
pao1_out_filename = "pao1_core_similarity_associations.tsv"
pa14_out_filename = "pa14_core_similarity_associations.tsv"

In [3]:
# Load transcriptional similarity df
pao1_similarity_scores_filename = "pao1_core_similarity_expression_stats.tsv"
pa14_similarity_scores_filename = "pa14_core_similarity_expression_stats.tsv"

pao1_similarity_scores = pd.read_csv(
    pao1_similarity_scores_filename, sep="\t", header=0, index_col=0
)
pa14_similarity_scores = pd.read_csv(
    pa14_similarity_scores_filename, sep="\t", header=0, index_col=0
)

In [4]:
print(pao1_similarity_scores.shape)
pao1_similarity_scores.head()

(5349, 11)


,PA14 homolog id,Transcriptional similarity across strains,P-value,Name,label,median expression,mean expression,variance expression,max expression,min expression,range expression
PAO1 id,,,,,,,,,,,
PA4275,PA14_08710,0.857417,0.000000e+00,nusG,most stable,2050.662908,2954.759407,9.010696e+06,9.010696e+06,58.690467,9.010638e+06
PA2393,PA14_33730,0.551535,0.000000e+00,NaN,most stable,68.382442,414.844592,9.981913e+05,9.981913e+05,1.083906,9.981902e+05
PA4025,PA14_11760,0.761600,0.000000e+00,NaN,most stable,105.678427,140.514545,2.172842e+04,2.172842e+04,0.641440,2.172777e+04
PA4722,PA14_62480,0.146158,6.268364e-27,NaN,least stable,318.711508,363.589133,4.043287e+04,4.043287e+04,39.492371,4.039338e+04
PA5360,PA14_70750,0.409206,4.399719e-215,phoB,NaN,175.031558,272.866730,1.558178e+05,1.558178e+05,0.713117,1.558170e+05


In [5]:
print(pa14_similarity_scores.shape)
pa14_similarity_scores.head()

(5348, 11)


,PAO1 homolog id,Transcriptional similarity across strains,P-value,Name,label,median expression,mean expression,variance expression,max expression,min expression,range expression
PA14 id,,,,,,,,,,,
PA14_68200,PA5163,0.715084,0.0,rmlA,most stable,1160.357075,1271.443599,478850.471118,478850.471118,107.537497,478742.933621
PA14_69780,PA5286,0.630547,0.0,NaN,most stable,232.872707,397.012374,239723.196897,239723.196897,15.377551,239707.819346
PA14_55200,PA0704,0.793462,0.0,NaN,most stable,118.752808,158.421448,18302.600541,18302.600541,10.044320,18292.556222
PA14_42320,PA1718,0.550805,0.0,pscE,most stable,32.409364,66.537502,12703.297406,12703.297406,1.659510,12701.637895
PA14_14700,PA3816,0.748091,0.0,cysE,most stable,375.793504,410.182084,39887.632007,39887.632007,72.337808,39815.294198


In [6]:
# Load KEGG pathway data
pao1_pathway_filename = "https://raw.githubusercontent.com/greenelab/adage/7a4eda39d360b224268921dc1f2c14b32788ab16/Node_interpretation/pseudomonas_KEGG_terms.txt"

In [7]:
pao1_pathways = annotations.load_format_KEGG(pao1_pathway_filename)
pao1_pathways.head()

,1,2
0,,
KEGG-Pathway-pae00072: Synthesis and degradation of ketone bodies,10,"{PA1999, PA3925, PA2553, PA1736, PA2003, PA478..."
KEGG-Pathway-pae00071: Fatty acid degradation,32,"{PA4899, PA2574, PA3426, PA1535, PA4435, PA044..."
KEGG-Pathway-pae00903: Limonene and pinene degradation,9,"{PA4899, PA1737, PA3426, PA1821, PA2475, PA333..."
KEGG-Pathway-pae00380: Tryptophan metabolism,27,"{PA4899, PA2147, PA3426, PA0447, PA4613, PA434..."
KEGG-Pathway-pae00900: Terpenoid backbone biosynthesis,16,"{PA3803, PA3925, PA4044, PA2553, PA1736, PA362..."


## Pathway annotations to PA14

The annotations we have are only for PAO1 genes, so we will map PAO1 core genes to PA14 core genes to add annotations to PA14. This is possible since we are focused on only core genes, which have homologs between PAO1 and PA14

In [8]:
pao1_annotation_filename = paths.GENE_PAO1_ANNOT
gene_mapping_pao1 = utils.get_pao1_pa14_gene_map(pao1_annotation_filename, "pao1")

In [9]:
gene_mapping_pao1 = gene_mapping_pao1["PA14_ID"].to_frame()

## Get pathway associations for all genes

In [10]:
def get_associated_pathways(genes_):
    rows = []
    for gene_id in genes_:
        pathway_bool = [
            gene_id in pao1_pathways.loc[pathway, 2] for pathway in pao1_pathways.index
        ]
        found_pathways = list(pao1_pathways[pathway_bool].index)
        rows.append({"gene id": gene_id, "pathways present": found_pathways})
    return pd.DataFrame(rows).set_index("gene id")

In [11]:
# Get KEGG associations for all genes in PAO1
all_pao1_gene_ids = list(pao1_similarity_scores.index)
pao1_associations = get_associated_pathways(all_pao1_gene_ids)

In [12]:
print(pao1_associations.shape)
pao1_associations.head()

(5349, 1)


,pathways present
gene id,
PA4275,[]
PA2393,[]
PA4025,[KEGG-Pathway-pae00564: Glycerophospholipid me...
PA4722,[]
PA5360,[]


In [13]:
# Map PA14 gene ids
pa14_associations = pao1_associations.merge(
    gene_mapping_pao1, left_index=True, right_index=True
)
pa14_associations.set_index("PA14_ID", inplace=True)
print(pa14_associations.shape)
pa14_associations.head()

(5349, 1)


,pathways present
PA14_ID,
PA14_08710,[]
PA14_33730,[]
PA14_11760,[KEGG-Pathway-pae00564: Glycerophospholipid me...
PA14_62480,[]
PA14_70750,[]


In [14]:
# Merge KEGG associations with transcriptional similarity information
pao1_associations = pao1_similarity_scores.merge(
    pao1_associations, left_index=True, right_index=True, how="left"
)
pa14_associations = pa14_similarity_scores.merge(
    pa14_associations, left_index=True, right_index=True, how="left"
)

In [15]:
print(pao1_associations.shape)
pao1_associations.head()

(5349, 12)


,PA14 homolog id,Transcriptional similarity across strains,P-value,Name,label,median expression,mean expression,variance expression,max expression,min expression,range expression,pathways present
PAO1 id,,,,,,,,,,,,
PA4275,PA14_08710,0.857417,0.000000e+00,nusG,most stable,2050.662908,2954.759407,9.010696e+06,9.010696e+06,58.690467,9.010638e+06,[]
PA2393,PA14_33730,0.551535,0.000000e+00,NaN,most stable,68.382442,414.844592,9.981913e+05,9.981913e+05,1.083906,9.981902e+05,[]
PA4025,PA14_11760,0.761600,0.000000e+00,NaN,most stable,105.678427,140.514545,2.172842e+04,2.172842e+04,0.641440,2.172777e+04,[KEGG-Pathway-pae00564: Glycerophospholipid me...
PA4722,PA14_62480,0.146158,6.268364e-27,NaN,least stable,318.711508,363.589133,4.043287e+04,4.043287e+04,39.492371,4.039338e+04,[]
PA5360,PA14_70750,0.409206,4.399719e-215,phoB,NaN,175.031558,272.866730,1.558178e+05,1.558178e+05,0.713117,1.558170e+05,[]


In [16]:
print(pa14_associations.shape)
pa14_associations.head()

(5348, 12)


,PAO1 homolog id,Transcriptional similarity across strains,P-value,Name,label,median expression,mean expression,variance expression,max expression,min expression,range expression,pathways present
PA14 id,,,,,,,,,,,,
PA14_68200,PA5163,0.715084,0.0,rmlA,most stable,1160.357075,1271.443599,478850.471118,478850.471118,107.537497,478742.933621,[KEGG-Pathway-pae00521: Streptomycin biosynthe...
PA14_69780,PA5286,0.630547,0.0,NaN,most stable,232.872707,397.012374,239723.196897,239723.196897,15.377551,239707.819346,[]
PA14_55200,PA0704,0.793462,0.0,NaN,most stable,118.752808,158.421448,18302.600541,18302.600541,10.044320,18292.556222,"[KEGG-Pathway-pae00380: Tryptophan metabolism,..."
PA14_42320,PA1718,0.550805,0.0,pscE,most stable,32.409364,66.537502,12703.297406,12703.297406,1.659510,12701.637895,[]
PA14_14700,PA3816,0.748091,0.0,cysE,most stable,375.793504,410.182084,39887.632007,39887.632007,72.337808,39815.294198,[KEGG-Pathway-pae00270: Cysteine and methionin...


In [17]:
# Save
pao1_associations.to_csv(pao1_out_filename, sep="\t")
pa14_associations.to_csv(pa14_out_filename, sep="\t")

**Takeaway:**

Based on the pathways associated with the most and least stable core genes, we find that the most stable core genes tend to be associated with pathways related to cellular maintenance including: protein transport systems, ribosomes, metabolism, type III, IV secretion system which mediates virulence.

There are far fewer KEGG pathways that least stable core genes are found to be associated with. The least stable core genes are mostly associated with different types of metabolism.

A google doc containing the most and least stable core genes and some information about them is [here](https://docs.google.com/spreadsheets/d/1SqEyBvutfbsOTo4afg9GiEzP32ZKplkN1a6MpAQBvZI/edit?usp=sharing).